# Sound Field Analysis

[return to main page](index.ipynb)

## Preparations

In [ ]:
import tools
import numpy as np
import sounddevice as sd  # for playback
import soundfile as sf  # for reading a soundfile
from scipy import signal

# remove "inline" to get a separate plotting window:
%matplotlib inline  
import matplotlib.pyplot as plt

# Linear Microphone Arrays

## Delay-and-Sum Beamformer

### Repetition: Spatial-Temporal Fourier Transform of a Plane Wave

We already got to know the temporal spectrum of a [plane wave](physics_of_sound_I-solutions.ipynb#Plane-Wave) with a direction of propagation $\mathbf n_{\mathrm {pw}}$ which reads

$$P_{pw}(\mathbf x, \omega) = \mathrm{exp}\left(-j\frac{\omega}{c} \mathbf n_{\mathrm {pw}} \cdot \mathbf x \right) = \mathrm{exp}\left(-j\frac{\omega}{c} (n_{\mathrm {pw},x} \cdot x + n_{\mathrm {pw},y} \cdot y + n_{\mathrm {pw},z} \cdot z) \right) \,. $$

As we already have the temporal spectrum of the plane wave we only need to calculate the three integral we respect to the space dimensions:

$$ \tilde P_{pw}(\mathbf k, \omega) = \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} P_{pw}(\mathbf x, \omega) e^{+jk_x x} e^{+jk_y y} e^{+jk_z z} \,\mathrm d x\,\mathrm d y\,\mathrm d z
$$

Inserting the spectrum of the plane wave
$$
\tilde P_{pw}(\mathbf k, \omega) = \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} e^{-j\frac{\omega}{c} (n_{\mathrm {pw},x} \cdot x + n_{\mathrm {pw},y} \cdot y + n_{\mathrm {pw},z} \cdot z)} e^{+jk_x x} e^{+jk_y y} e^{+jk_z z} \,\mathrm d x\,\mathrm d y\,\mathrm d z
$$

Splitting up the exponentials and re-arranging the integrals yields

$$
\tilde P_{pw}(\mathbf k, \omega) = 
    \int_{-\infty}^{\infty} \mathrm{exp}\left(+j\left[k_x-\frac{\omega}{c} n_{\mathrm {pw},x}\right]x\right) \,\mathrm d x \cdot
    \int_{-\infty}^{\infty} \mathrm{exp}\left(+j\left[k_y-\frac{\omega}{c} n_{\mathrm {pw},y}\right]y\right) \,\mathrm d y \cdot
    \int_{-\infty}^{\infty} \mathrm{exp}\left(+j\left[k_z-\frac{\omega}{c} n_{\mathrm {pw},z}\right]z\right) \,\mathrm d z
$$

Using the integral indentity

$$ \int_{-\infty}^{\infty} e^{+j a b}\,\mathrm d b
    = \int_{-\infty}^{\infty} \mathrm{exp}\left(+j a b\right)\,\mathrm d b 
    = 2\pi \delta(a) 
$$

for each of the integrals yields

$$
\tilde P_{pw}(\mathbf k, \omega) = 
    2\pi \delta\left(k_x-\frac{\omega}{c} n_{\mathrm {pw},x}\right) \cdot
    2\pi \delta\left(k_y-\frac{\omega}{c} n_{\mathrm {pw},y}\right) \cdot
    2\pi \delta\left(k_z-\frac{\omega}{c} n_{\mathrm {pw},z}\right)
    =
    (2\pi)^3 \delta\left(\mathbf k -\frac{\omega}{c} \mathbf n_{\mathrm {pw}}\right)
$$


In [ ]:
omega = 2*np.pi * np.linspace(20, 8000, 1000)  # evaluated angular frequencies
theta_pw = np.linspace(0, np.pi, 181)  # evaluated incident plane waves

def compute_dsb_beampattern(theta, theta_pw, omega, dx, M, c=343):
    X_pw = np.zeros(shape=(len(omega), len(theta_pw)), dtype=complex)
    for n, k in enumerate(omega/c):
        for dm in np.arange(M)*dx:
            X_pw[n, :] += np.exp(-1j * k * dm * (np.cos(theta_pw) - np.cos(theta)))
            
    return X_pw/M

def plot_dsb_beampattern(X_pw, theta_pw, omega):
    
    X_pw = X_pw / np.max(np.abs(X_pw[:]))
    
    plt.figure(figsize=(10,10))
    plt.pcolormesh(theta_pw/np.pi*180, omega/(2*np.pi), 20*np.log10(np.abs(X_pw)), cmap='viridis')
    plt.colormaps()
    plt.ylim([0, 8000])
    plt.xlim([0, 180])
    plt.colorbar()
    plt.clim([-40,0])

    plt.xlabel(r'$\theta_{pw}$ in deg')
    plt.ylabel('$f$ in Hz')
    plt.title(r'$|\bar{P}(\theta, \theta_{pw}, \omega)|$ in dB')

### Infinite, Discrete Array

In [ ]:
dx = 0.12 # distance between microphones
M = 1000  # total number of microphones (approx. infinite)
theta = np.pi/2  # steering direction

X_pw = compute_dsb_beampattern(theta, theta_pw, omega, dx, M)
plot_dsb_beampattern(X_pw, theta_pw, omega)

### Truncated, Discrete Array

In [ ]:
dx = 0.12 # distance between microphones
M = 8 # total number of microphones (approx. infinite)
theta = np.pi/2  # steering direction

X_pw = compute_dsb_beampattern(theta, theta_pw, omega, dx, M)
plot_dsb_beampattern(X_pw, theta_pw, omega)

### A real Array in a real Room

In [ ]:
# sd.query_devices()
sd.default.device = 4  # Hammerfall DSP
output_channels = [1,2]  # first two analog outputs
input_channels = np.arange(9,17)  # all 8 ADAT input channels

[x_target, fs] = sf.read('data/xmas.wav')  # target sound
x_noise = 0.005*np.random.randn(len(x_target))  # gaussian noise as masker

def dsb_beamforming(x_rec, theta, dx, M, nfft, c=343):
    # get temporal spectrum (DFT)
    X_rec = np.fft.rfft(x_rec, n=nfft, axis=0)
    omega = 2*np.pi*np.fft.rfftfreq(nfft, 1/fs)
    
    # get plane wave decomposition using DSB
    X_pw = np.zeros((len(omega),len(theta)), dtype='complex')
    for tdx, th in enumerate(theta):
        for ddx, dm in enumerate(np.arange(M)*dx):
            X_pw[:,tdx] += X_rec[:,ddx]*np.exp(-1j*omega/c*dm*np.cos(th))
    
    return X_pw/M, omega

#### One Source

In [ ]:
# record signal 1
x_rec_one = sd.playrec(x_noise, 
                   samplerate=fs,
                   input_mapping=input_channels, 
                   output_mapping=1, 
                   blocking=True)

# record signal 2
x_rec_two = sd.playrec(x_noise, 
                   samplerate=fs,
                   input_mapping=input_channels, 
                   output_mapping=2, 
                   blocking=True)

In [ ]:
dx = 0.12 # distance between microphones
M = 8 # total number of microphones (approx. infinite)
theta = np.linspace(0, np.pi, 181) # steering directions

X_pw_one, omega = dsb_beamforming(x_rec_one, theta, dx, M, 2*len(x_rec_one))
plot_dsb_beampattern(X_pw_one, theta_pw, omega)

In [ ]:
dx = 0.12 # distance between microphones
M = 8 # total number of microphones (approx. infinite)
theta = np.linspace(0, np.pi, 181) # steering directions

X_pw_two, omega = dsb_beamforming(x_rec_one, theta, dx, M, 2*len(x_rec_two))
plot_dsb_beampattern(X_pw_two, theta_pw, omega)

<p xmlns:dct="http://purl.org/dc/terms/">
  <a rel="license"
     href="http://creativecommons.org/publicdomain/zero/1.0/">
    <img src="http://i.creativecommons.org/p/zero/1.0/88x31.png" style="border-style: none;" alt="CC0" />
  </a>
  <br />
  To the extent possible under law,
  <span rel="dct:publisher" resource="[_:publisher]">the person who associated CC0</span>
  with this work has waived all copyright and related or neighboring
  rights to this work.
</p>